### Imports 

In [ ]:
from langchain_neo4j import Neo4jVector
from langchain_openai import OpenAIEmbeddings
import os 
from dotenv import load_dotenv
import json 
from src.CypherGenerator.cypher_generator import CypherGenerator, CypherUtils
from src.CypherGenerator.models.data_model import DocumentData, ReasoningStep, QA
from neo4j import GraphDatabase
load_dotenv()

### Cypher spliter and executor function  

In [ ]:

# LLM and Database clients
driver = GraphDatabase.driver(uri=os.getenv("Neo4j_URL"), auth=(os.getenv("Neo4j_USERNAME"), os.getenv("Neo4j_PASSWORD")))


In [ ]:
def ingest_to_neo4j(cypher_script):
    with driver.session() as session:
        # Split by ';' to execute each statement separately, ignore empty
        for stmt in cypher_script.split(';'):
            stmt = stmt.strip()
            if stmt:
                session.run(stmt)

### Ingest into Neo4j Database

In [ ]:
with open('src/data/train.json') as f:
    raw_data = json.load(f)

for doc in raw_data:
    document_data = DocumentData(**doc)

    # Create the generator instance correctly with DocumentData
    cypher_generator = CypherGenerator(document_data)
    # Generate Cypher statements
    try:
        cypher_statements = cypher_generator.generate()
        ingest_to_neo4j(cypher_statements)
    except Exception as e:
        if 'error_list' not in locals():
            error_list = []
        error_list.append({"doc_id": document_data.id, "error": str(e)})
    # Output statements
    print(error_list)

### Vectorise relevent nodes

In [ ]:
# Create the vectorstore for our existing graph
cell_graph = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY")),
    url=os.getenv("Neo4j_URL"),
    username=os.getenv("Neo4j_USERNAME"),
    password=os.getenv("Neo4j_PASSWORD"),
    index_name="vinx_cellval",
    node_label="Cell",
    text_node_properties=["value"],
    embedding_node_property="cell_embedding",
)

In [ ]:
# Create the vectorstore for our existing graph
Sent_graph = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY")),
    url=os.getenv("Neo4j_URL"),
    username=os.getenv("Neo4j_USERNAME"),
    password=os.getenv("Neo4j_PASSWORD"),
    index_name="vinx_sentval",
    node_label="Sentence",
    text_node_properties=["text"],
    embedding_node_property="cell_embedding",)